#Step 1: Simulate Rankings of Relevance for E and P

In the first step generate pairs of rankings, for the production P and experimental E, respectively. Assume a binary relevance. Make no assumption regarding the documents returned by the two algorithms (they can be distinct but they may also overlap). Further, assume that the algorithms are used on mobiles, so we are interested only in rankings of length 3. 



In [4]:
import itertools

def produce_rank(labels = [0, 1] , n=3):
    return list(map(list, itertools.product(labels, repeat=n)))


def creat_rank_pairs(labels = [0, 1] , n=3):
    rank_e = produce_rank(labels = [0, 1] , n=3)
    rank_p = produce_rank(labels = [0, 1] , n=3)

    rank_pairs = []

    for re in rank_e:
        for rp in rank_p:
            rank_pairs.append([rp, re])
            
    return rank_pair
            
rank_pair = creat_rank_pairs
print(rank_pairs)               

[[[0, 0, 0], [0, 0, 0]], [[0, 0, 1], [0, 0, 0]], [[0, 1, 0], [0, 0, 0]], [[0, 1, 1], [0, 0, 0]], [[1, 0, 0], [0, 0, 0]], [[1, 0, 1], [0, 0, 0]], [[1, 1, 0], [0, 0, 0]], [[1, 1, 1], [0, 0, 0]], [[0, 0, 0], [0, 0, 1]], [[0, 0, 1], [0, 0, 1]], [[0, 1, 0], [0, 0, 1]], [[0, 1, 1], [0, 0, 1]], [[1, 0, 0], [0, 0, 1]], [[1, 0, 1], [0, 0, 1]], [[1, 1, 0], [0, 0, 1]], [[1, 1, 1], [0, 0, 1]], [[0, 0, 0], [0, 1, 0]], [[0, 0, 1], [0, 1, 0]], [[0, 1, 0], [0, 1, 0]], [[0, 1, 1], [0, 1, 0]], [[1, 0, 0], [0, 1, 0]], [[1, 0, 1], [0, 1, 0]], [[1, 1, 0], [0, 1, 0]], [[1, 1, 1], [0, 1, 0]], [[0, 0, 0], [0, 1, 1]], [[0, 0, 1], [0, 1, 1]], [[0, 1, 0], [0, 1, 1]], [[0, 1, 1], [0, 1, 1]], [[1, 0, 0], [0, 1, 1]], [[1, 0, 1], [0, 1, 1]], [[1, 1, 0], [0, 1, 1]], [[1, 1, 1], [0, 1, 1]], [[0, 0, 0], [1, 0, 0]], [[0, 0, 1], [1, 0, 0]], [[0, 1, 0], [1, 0, 0]], [[0, 1, 1], [1, 0, 0]], [[1, 0, 0], [1, 0, 0]], [[1, 0, 1], [1, 0, 0]], [[1, 1, 0], [1, 0, 0]], [[1, 1, 1], [1, 0, 0]], [[0, 0, 0], [1, 0, 1]], [[0, 0, 1], [1,